In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import env
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats

from math import sqrt

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# imports for modeling:
# import Logistic regression
from sklearn.linear_model import LogisticRegression
# import K Nearest neighbors:
from sklearn.neighbors import KNeighborsClassifier
# import Decision Trees:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# import Random Forest:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

# interpreting our models:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#feature selection
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector


<h1>Feature Engineering Exercises<h1/>

<h3>Load the 'tips' dataset.

<h3/>

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


<h3>a. Create a column named price_per_person. This should be the total bill divided by the party size.
<h3/>

In [4]:
df['price_per_person'] = df.total_bill / df.size
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,0.009947
1,10.34,1.66,Male,No,Sun,Dinner,3,0.006054
2,21.01,3.50,Male,No,Sun,Dinner,3,0.012301
3,23.68,3.31,Male,No,Sun,Dinner,2,0.013864
4,24.59,3.61,Female,No,Sun,Dinner,4,0.014397


<h3>b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
<h3/>

In [5]:
# size, total_bill

<h3>c. Use Select K Best to select the top 2 features for predicting tip amount. What are they?
<h3/>

In [6]:
df.head(2)

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,0.009947
1,10.34,1.66,Male,No,Sun,Dinner,3,0.006054


In [7]:
def clean_tips(df):
    dummy_df = pd.get_dummies(df[['sex', 'day', 'time', 'smoker']], drop_first=False)
    cleaned_df = pd.concat([df, dummy_df], axis = 1)
    return cleaned_df

def machine_tips(cleaned_df):
    machine_df = cleaned_df.drop(columns = ['sex', 'day', 'time', 'smoker'])
    return machine_df

In [8]:
cleaned_df = clean_tips(df)
cleaned_df.head(2)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No
0,16.99,1.01,Female,No,Sun,Dinner,2,0.009947,0,1,0,0,0,1,0,1,0,1
1,10.34,1.66,Male,No,Sun,Dinner,3,0.006054,1,0,0,0,0,1,0,1,0,1


In [9]:
machine_df = machine_tips(cleaned_df)
machine_df.head(2)

,total_bill,tip,size,price_per_person,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No
0,16.99,1.01,2,0.009947,0,1,0,0,0,1,0,1,0,1
1,10.34,1.66,3,0.006054,1,0,0,0,0,1,0,1,0,1


In [10]:
def split_data(machine_df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    
    # Create train_validate and test datasets
    train_validate, test = train_test_split(machine_df, test_size=0.2, random_state=123)
    
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=123)

    # Take a look at your split datasets

    print(f"""
    train -> {train.shape}
    validate -> {validate.shape}
    test -> {test.shape}""")
    
    return train, validate, test

In [11]:
train, validate, test = split_data(machine_df)
train.head()


    train -> (136, 14)
    validate -> (59, 14)
    test -> (49, 14)


,total_bill,tip,size,price_per_person,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,smoker_Yes,smoker_No
18,16.97,3.50,3,0.009936,0,1,0,0,0,1,0,1,0,1
172,7.25,5.15,2,0.004245,1,0,0,0,0,1,0,1,1,0
118,12.43,1.80,2,0.007278,0,1,1,0,0,0,1,0,0,1
28,21.70,4.30,2,0.012705,1,0,0,0,1,0,0,1,0,1
237,32.83,1.17,2,0.019221,1,0,0,0,1,0,0,1,1,0


In [12]:
X_train=train.drop(columns='tip')
y_train=train.tip

X_validate=validate.drop(columns='tip')
y_validate=validate.tip

X_test=test.drop(columns='tip')
y_test=test.tip

In [13]:
# MAKE the thing
kbest = SelectKBest(f_regression, k=2)

# FIT the thing
kbest.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x148a74a40>)

In [14]:
# statistical f-value / feature's scores:
kbest.scores_

array([1.15984909e+02, 6.12590886e+01, 1.15984909e+02, 1.15479204e+00,
       1.15479204e+00, 1.07637964e+00, 6.92417587e-01, 3.18953072e-03,
       2.40040378e+00, 1.79864746e+00, 1.79864746e+00, 3.44908618e-01,
       3.44908618e-01])

In [15]:
# p value: 
kbest.pvalues_

array([7.18647033e-20, 1.34164190e-12, 7.18647033e-20, 2.84479443e-01,
       2.84479443e-01, 3.01377404e-01, 4.06823770e-01, 9.55046793e-01,
       1.23662519e-01, 1.82144884e-01, 1.82144884e-01, 5.57997755e-01,
       5.57997755e-01])

In [16]:
kbest.feature_names_in_

array(['total_bill', 'size', 'price_per_person', 'sex_Male', 'sex_Female',
       'day_Thur', 'day_Fri', 'day_Sat', 'day_Sun', 'time_Lunch',
       'time_Dinner', 'smoker_Yes', 'smoker_No'], dtype=object)

In [17]:
kbest_results = pd.DataFrame(
                dict(p=kbest.pvalues_, f=kbest.scores_),
                                        index = X_train.columns)

In [18]:
kbest_results

,p,f
total_bill,7.186470e-20,115.984909
size,1.341642e-12,61.259089
price_per_person,7.186470e-20,115.984909
sex_Male,2.844794e-01,1.154792
sex_Female,2.844794e-01,1.154792
day_Thur,3.013774e-01,1.076380
day_Fri,4.068238e-01,0.692418
day_Sat,9.550468e-01,0.003190
day_Sun,1.236625e-01,2.400404
time_Lunch,1.821449e-01,1.798647


In [19]:
# get-support() will output a boolean mask to tell me which features were selected
kbest.get_support()

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False, False])

In [20]:
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]

Index(['total_bill', 'price_per_person'], dtype='object')

In [21]:
# Let's turn the KB-selected features into a DF
X_train_KBtransformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)

In [22]:
# The results of KB transformed and ready for modeling if these were the only two features we wanted to look at

X_train_KBtransformed.head()

,total_bill,price_per_person
18,16.97,0.009936
172,7.25,0.004245
118,12.43,0.007278
28,21.70,0.012705
237,32.83,0.019221


<h3>d. Use Recursive Feature Elimination to select the top 2 features for tip amount. What are they?
<h3/>

In [24]:
from sklearn.linear_model import LinearRegression

# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space

model = LinearRegression()

In [25]:
# MAKE the thing
rfe = RFE(model, n_features_to_select=2)

# FIT the thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [26]:
# Get feature ranking
# Selected features are assigned a rank 1

rfe.ranking_

array([ 7,  1, 12,  2,  3,  1,  6,  5,  9, 10, 11,  8,  4])

In [28]:
# Dataframe of rankings

pd.DataFrame(
{
    'rfe_ranking':rfe.ranking_
}, index = X_train.columns)

,rfe_ranking
total_bill,7
size,1
price_per_person,12
sex_Male,2
sex_Female,3
day_Thur,1
day_Fri,6
day_Sat,5
day_Sun,9
time_Lunch,10


In [29]:
X_train_RFEtransformed = pd.DataFrame(
                            rfe.transform(X_train),
                            index = X_train.index,
                            columns = X_train.columns[rfe.support_])

In [30]:
X_train_RFEtransformed.head()

,size,day_Thur
18,3.0,0.0
172,2.0,0.0
118,2.0,1.0
28,2.0,0.0
237,2.0,0.0


<h3>e. Why do you think Select K Best and Recursive Feature Elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?
<h3/>

<h4>**Answer: I think I got different answers for the top features because Select K Best is the fastest, but not the most accurate option, while RFE took its time and weighed day_Thurs as a better feature to predict tip amount**<h4/>

<h3>Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<h3/>

In [38]:
def select_kbest(X_train, y_train, k):
    # MAKE the thing
    kbest = SelectKBest(f_regression, k=k)

    # FIT the thing
    kbest.fit(X_train, y_train)
    return X_train.columns[kbest.get_support()]

In [40]:
select_kbest(X_train, y_train, 2)

Index(['total_bill', 'price_per_person'], dtype='object')

<h3>Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top n features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

<h3/>

In [41]:
def rfe(X_train, y_train, k):
    model = LinearRegression()
    
    # MAKE the thing
    rfe = RFE(model, n_features_to_select=k)
    
    # FIT the thing
    rfe.fit(X_train, y_train)
    
    X_train_RFEtransformed = pd.DataFrame(
                            rfe.transform(X_train),
                            index = X_train.index,
                            columns = X_train.columns[rfe.support_])
    
    return X_train_RFEtransformed.columns

In [42]:
rfe(X_train, y_train, 2)

Index(['size', 'day_Thur'], dtype='object')

<h3>Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both Select K Best and Recursive Feature Elimination (use the functions you just built to help you out).

<h3/>

In [36]:
swiss = pd.read_csv('swiss.csv')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [37]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [43]:
def split_data(swiss):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    
    # Create train_validate and test datasets
    train_validate, test = train_test_split(swiss, test_size=0.2, random_state=123)
    
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=123)

    # Take a look at your split datasets

    print(f"""
    train -> {train.shape}
    validate -> {validate.shape}
    test -> {test.shape}""")
    
    return train, validate, test

In [44]:
train, validate, test = split_data(swiss)
train.head()


    train -> (25, 6)
    validate -> (12, 6)
    test -> (10, 6)


,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.5,60.8,16,10,7.72,16.3
Lavaux,65.1,73.0,19,9,2.84,20.0
Nyone,56.6,50.9,22,12,15.14,16.7
Conthey,75.5,85.9,3,2,99.71,15.1
Yverdon,65.4,49.5,15,8,6.10,22.5


In [45]:
X_train=train.drop(columns='Fertility')
y_train=train.Fertility

X_validate=validate.drop(columns='Fertility')
y_validate=validate.Fertility

X_test=test.drop(columns='Fertility')
y_test=test.Fertility

In [46]:
#Select K Best - Top 3 results
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

In [47]:
#Recursive Feature Elimination - Top  3 results
rfe(X_train, y_train, 3)

Index(['Agriculture', 'Examination', 'Infant.Mortality'], dtype='object')